In [49]:
import flywheel
import pandas as pd
from datetime import datetime
import os 
import glob

fw = flywheel.Client()
gear = 'deid-export'
gear_fw = fw.lookup(f'gears/{gear}')
collection_id = '618d6fb6bb845b310730d8f8' # Neuroprint Validation collection
project_id = '60fef55e60ec55d1b0e0741e' # Neuroprint Validation project
project = fw.get(project_id)
collection = fw.get(collection_id)

## Run gear to move sessions

In [67]:
collection_sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [40]:
for ses in collection_sessions[0:2]:
    project = fw.get(session.parents['project'])
    deid_profile_list = [f for f in project.files if 'blank-export' in f.name]
    if len(deid_profile_list) > 0:
        inputs = {"deid_profile": deid_profile_list[0]}
        config = {"project_path": 'detre_group/Neuroprint Validation', 'overwrite_files': True}
        asys_label = 'deid-export_WT_NeuroprintValidation'
        analysis_id = gear_fw.run(analysis_label=asys_label, config=config, inputs=inputs, destination=ses)

In [41]:
# check status real quick
data_dict = {'subject':[],'session':[],'run':[]}
for ses in collection_sessions[0:2]:
    session = ses.reload()
    sub_label = session.subject.label
    ses_label = session.label
    analyses = session.analyses

    if len(analyses) == 0:
        run = 'False'
    else:
        # Loop through the analyses
        matches = [asys for asys in analyses if asys.gear_info.get('name') == gear]
        if len(matches) == 0:
            run = 'False'
        elif len(matches) == 1:
            run = 'True'
        else:
            run = 'True (multiple)'
            
    data_dict['subject'].append(sub_label)
    data_dict['session'].append(ses_label)
    data_dict['run'].append(run)

df = pd.DataFrame.from_dict(data_dict)
df

,subject,session,run
0,123894,123894x20190327x3T,True
1,100049,100049x20160621x3T,True


In [68]:
# delete gear runs (they'll still remain in provenance, but will be out of sight for other people)
for ses in collection_sessions[0:2]:
    for analysis in ses.analyses:
        if 'deid-export_WT_NeuroprintValidation' in analysis.label:
            fw.delete_session_analysis(ses.id, analysis.id)
            print(f"Deleted {analysis.label} from {ses.label}.")

Deleted deid-export_WT_NeuroprintValidation from 123894x20190327x3T.
Deleted deid-export_WT_NeuroprintValidation from 100049x20160621x3T.


## Remove non-T1 acquisitions

In [51]:
project_sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

In [61]:
delete_dict = {'session':[], 'acquisition':[],'status':[]}
for ses in project_sessions:
    acqlist = ses.acquisitions()
    for acq in acqlist:        
        lab = acq.label.lower()
        if ("vnav" in lab) and ("moco" in lab) and ("rms" in lab) and not ("nd" in lab):
            status = 'kept'
        elif ("vnav" in lab) and ("rms" in lab) and not ("nd" in lab):
            status = 'kept'
        elif ("ax" in lab) and ("mprage" in lab):
            status = 'kept'
        elif ("sag" in lab) and ("mprage" in lab):
            status = 'kept'
        elif ("t1_3d" in lab and not ("nd" in lab)): #NACC
            status = 'kept'
        else:
            fw.delete_acquisition(acq.id)
            status='deleted'

        delete_dict['session'].append(ses.label)
        delete_dict['acquisition'].append(acq.label)
        delete_dict['status'].append(status)
        

delete_df = pd.DataFrame.from_dict(delete_dict)
delete_df = delete_df.sort_values('status', ascending=False) # show what's kept at the top
delete_df.head()

,session,acquisition,status
21,123894x20190327x3T,T1_3D_0.8x0.8x0.8,kept
31,100049x20160621x3T,T1_3D_0.8x0.8x0.8,kept
0,123894x20190327x3T,AAHead_Scout_MPR_sag,deleted
36,100049x20160621x3T,TOF_3D_multi-slab_MIP_SAG,deleted
27,100049x20160621x3T,Mag_Images,deleted


In [18]:
# Append a timestamp to our csv name so it won't overwrite anything when we upload it to flywheel
time_fmt = '%m-%d-%Y_%H-%M-%S'
time_string = datetime.now().strftime(time_fmt)
csv_out = f'../run_reports/{gear}_DeletionReport_{time_string}.csv'

delete_df.to_csv(csv_out,index=False)

#project.upload_file(csv_out)

## Move ANTs analyses to new project

In [69]:
# download the analysis data from the collection
ants_gear = 'antsct-aging-fw'
for ses in collection_sessions[0:2]:
    adir = os.path.join('/media/will/Samsung_T5/store/analyses', ses.subject.label, ses.label)
    os.makedirs(adir,exist_ok=True)
    for asys in ses.analyses:
        if (asys.gear_info.get('name') == ants_gear) and (asys.job['state']=='complete') and (asys.files):
            # select file and download
            antsct_output = [f for f in asys.files if f.name.endswith('zip')][0].name
            asys.download_file(antsct_output, f'{adir}/{antsct_output}')
            # record label in text output
            with open(f'{adir}/label.txt', 'w') as f:
                f.write(asys.label)


In [ ]:
# create analyses in Flywheel project
for ses in project_sessions:
    # get the input T1 file, which should already be in the session's acquisitions
    acq = ses.acquisitions()[0] # should only be one after deletion
    nii = [f for f in acq.files if f.name.endswith('.nii.gz')][0]
    file_ref = acq.get_file(nii.name).ref()
    print(file_ref)    
    
    # create the analysis container 
    adir = os.path.join('/media/will/Samsung_T5/store/analyses', ses.subject.label, ses.label)
    try:
        with open (f"{adir}/label.txt", "r") as label: # read the label text
            lab=label.readlines()
        os.remove(f"{adir}/label.txt")
    except FileNotFoundError as e:
        print(e)
        continue
        
    asys_label = lab[0]
    asys = ses.add_analysis(label=asys_label,inputs=[file_ref])
    
    # upload the results zip file 
    zip_file = glob.glob(adir+"/*.zip")[0]
    print(zip_file)
    asys.upload_output(zip_file)


{'type': 'acquisition', 'id': '619537efdd46e7e76fc3b392', 'name': '1.3.12.2.1107.5.2.43.66044.2019032710053479167074791.0.0.0.nii.gz'}
/media/will/Samsung_T5/store/analyses/123894/123894x20190327x3T/antsct_sub-123894_ses-123894x20190327x3T.zip
